In [8]:
library(ggplot2)
library(caTools)
library(haven)
library(readr)
library(tidyverse)
library(dplyr)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ stringr   1.5.1
✔ forcats   1.0.0     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


# Real Data

## 3A

In [15]:
setwd("C:/Users/VICTOR/Documents/GitHub/High_Dimensional_Linear_Models")  

In [16]:
df <- read.csv("R/Input/apartments.csv")       

In [18]:
head(df)

,id,price,month,area,type,rooms,centredistance,schooldistance,clinicdistance,postofficedistance,⋯,restaurantdistance,collegedistance,pharmacydistance,ownership,buildingmaterial,hasparkingspace,hasbalcony,haselevator,hassecurity,hasstorageroom
,<chr>,<int>,<int>,<int>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,a01d82c9529f98a54d64b9e061c9a73b,1199999,1,105,apartmentBuilding,4,5.06,1.080,0.949,0.623,⋯,1.054,3.062,0.335,condominium,brick,no,yes,yes,no,no
2,8373aa373dbc3fe7ca3b7434166b8766,650000,1,73,tenement,3,3.24,0.275,0.672,0.367,⋯,0.300,1.857,0.280,condominium,brick,no,no,no,no,no
3,7d0c31d5409caab173571cce3dcdf702,590000,1,69,blockOfFlats,3,3.94,0.139,1.336,0.926,⋯,0.071,0.786,0.304,condominium,brick,no,yes,no,no,yes
4,3eaa36a59b9354206703b5f6b2f2ff1d,584999,1,42,blockOfFlats,2,5.19,0.209,1.533,0.201,⋯,0.157,2.722,0.257,condominium,,no,yes,no,no,no
5,027b30cebbc49faf3094421b741ddd56,363000,1,46,blockOfFlats,2,1.65,0.250,0.624,0.510,⋯,0.232,0.986,0.273,condominium,concreteSlab,no,no,no,no,yes
6,27437e173a8c37d5002c0bf69c848b7c,717000,1,81,tenement,3,1.76,0.083,0.257,0.294,⋯,0.102,0.256,0.301,condominium,brick,no,no,no,no,yes


In [34]:
df <- df %>% mutate(area2 = area^2)

In [23]:
df <- df %>%
  mutate(
    hasparkingspace = ifelse(hasparkingspace == "yes", 1, 0),
    hasbalcony      = ifelse(hasbalcony == "yes", 1, 0),
    haselevator     = ifelse(haselevator == "yes", 1, 0),
    hassecurity     = ifelse(hassecurity == "yes", 1, 0),
    hasstorageroom  = ifelse(hasstorageroom == "yes", 1, 0)
  )

In [26]:
df <- df %>%
  mutate(digito = area %% 10)
for (d in 0:9) {
  varname <- paste0("end_", d)
  df[[varname]] <- as.integer(df$digito == d)  
}

## 3B

In [29]:
# variables de distancias
distancias <- c("schooldistance","clinicdistance","postofficedistance", "kindergartendistance","restaurantdistance","collegedistance", 
                "pharmacydistance")
# variables de digitos
digitos <- paste0("end_", 0:8) 
# variables h
h_variables <- c("hasparkingspace","hasbalcony","haselevator","hassecurity","hasstorageroom")
# variables categoricas
cat_variables <- c("month","type","rooms","ownership","buildingmaterial")


In [30]:
regresores <- c(digitos, "area", "area2", distancias, h_variables,
               paste0("factor(", cat_variables, ")"))


In [32]:
formula_regresion <- paste("price ~", paste(regresores, collapse = " + "))
formula_full <- as.formula(formula_regresion) # verificar


In [36]:
model1 <- lm(formula_full, data = df)
summary(model1)


Call:
lm(formula = formula_full, data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-1395903  -168244    -9358   147485  1893597 

Coefficients:
                                       Estimate Std. Error t value Pr(>|t|)    
(Intercept)                           1.127e+05  8.835e+03  12.758  < 2e-16 ***
end_0                                 1.813e+04  4.023e+03   4.507 6.57e-06 ***
end_1                                -1.612e+04  4.372e+03  -3.688 0.000226 ***
end_2                                 7.850e+03  4.153e+03   1.890 0.058708 .  
end_3                                 1.812e+04  4.239e+03   4.275 1.91e-05 ***
end_4                                -2.223e+02  4.139e+03  -0.054 0.957166    
end_5                                 1.658e+04  4.162e+03   3.984 6.79e-05 ***
end_6                                 2.936e+03  4.074e+03   0.721 0.471098    
end_7                                 1.677e+04  4.079e+03   4.111 3.94e-05 ***
end_8                               

In [ ]:
## Comentario: El coeficiente de end_0 indica la diferencia de precio entre apartamentos cuyo area termina en 0
## y la categoría base (último dígito es 9) manteniendo constantes las demás covariables.

### Partialling-out

In [38]:
X_menos_d0 <- setdiff(regresores, "end_0")

formula_po <- as.formula(paste("price ~", paste(X_menos_d0, collapse = " + ")))
modelo_2 <- lm(formula_po, data = df)
r_y <- resid(modelo_2)

formula_po2 <- as.formula(paste("end_0 ~", paste(X_menos_d0, collapse = " + ")))
modelo3 <- lm(formula_po2, data = df)
r_d <- resid(modelo3)

regresion_po <- lm(r_y ~ r_d - 1)  
summary(regresion_po)



Call:
lm(formula = r_y ~ r_d - 1)

Residuals:
     Min       1Q   Median       3Q      Max 
-1395903  -168244    -9358   147485  1893597 

Coefficients:
    Estimate Std. Error t value Pr(>|t|)    
r_d    18131       4022   4.508 6.54e-06 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 287600 on 106933 degrees of freedom
Multiple R-squared:  0.00019,	Adjusted R-squared:  0.0001807 
F-statistic: 20.32 on 1 and 106933 DF,  p-value: 6.543e-06


In [39]:
coef_end0 <- coef(summary(model1))["end_0","Estimate"]
coef_po_end0 <- coef(summary(regresion_po))["r_d","Estimate"]


In [40]:
print(coef_end0)
print(coef_po_end0)

[1] 18131.22
[1] 18131.22


## 3C